In [ ]:
% matplotlib inline

import numpy as np
from numpy import linalg, random, ones, zeros, matrix, eye, dot
from numpy.linalg import norm, cholesky, inv
from sklearn.cross_validation import train_test_split
import mosek
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import sys
import time
import scipy
from collections import namedtuple

v = .00001
delta = 0.01
sigma = .01
initial_rho = 1
max_iter = 100
initial_step_size = .1
timer_thresh = .1
ep = .0001
points_count = 100
points_std_from_surface = 0.0001
# D = 10000

def kernel(x1, x2):
    return math.exp(-1 * math.pow(norm(x1 - x2), 2
                                  ) / (2 * math.pow(sigma, 2)))

def kernel_vect(x_list, x2):
    return np.exp(-1 * np.power(norm(x_list - x2, axis=1), 2) / (2 * math.pow(sigma, 2)))

def get_K():
    start = time.time()

    K = np.zeros((g_m,g_m))
    for i in range(g_m):
        K[i, :] = kernel_vect(g_x, g_x[i])
                
    end = time.time()
    if end - start > timer_thresh:
        print 'get_K:', end - start, 'sec'
    return K

def get_data_points():
    start = time.time()
    points = random.random((points_count, 2)) * 2 * np.pi

    x = np.zeros((points_count, 3))
    for p in range(points_count):
        if points_std_from_surface > 0:
            r = random.normal(loc=1, scale=points_std_from_surface)
        else:
            r = 1
        z_cord = r * np.sin(points[p][1])

        r_temp = r * np.cos(points[p][1])
        y_cord = r_temp * np.sin(points[p][0])
        x_cord = r_temp * np.cos(points[p][0])

        x[p] = np.asarray([x_cord, y_cord, z_cord])

    end = time.time()
    if end - start > timer_thresh:
        print 'get_data_points:', end - start, 'sec'
    return x

def LU_decomp(A):
    start = time.time()
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    for k in range(n-1):
        assert A[k, k] != 0
        if A[k,k] == 0:
            print 'Error: Null Pivot'
            return 'Error: Null Pivot'
        A[k,k]=np.sqrt(A[k,k])
        A[k+1:n,k]=A[k+1:n,k]/A[k,k]
        for j in range(k+1,n):
            A[j:n,j]=A[j:n,j]-A[j:n,k]*A[j,k]
    A[n-1,n-1]=np.sqrt(A[n-1,n-1])
    A = np.tril(A)
    A = A.T
    end = time.time()
#     if end - start > timer_thresh:
    print 'LU_decomp:', end - start, 'sec'
    return A

def incomplete_LU_decomp(A,L):
    start = time.time()

    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
        
    for i in range(n):
        assert A[i,i] != 0
        L[i,i] = math.sqrt(A[i,i]-( np.sum(np.power(L[i,0:i-1],2)) ))
        assert L[i,i] != 0
        L[i,i+1:n] = 1/L[i,i]*(A[i,i+1:n]-np.dot(L[i+1:n,0:i-1],L[i,0:i-1]))
        
    end = time.time()
#     if end - start > timer_thresh:
    print 'incomplete_LU_decomp:', end - start, 'sec'
    return L


for i in range(8):
    print
    print 'points_count',points_count
    g_x = get_data_points()
    g_m = len(g_x)

    # fig = plt.figure(figsize=(10, 12))
    # ax = fig.add_subplot(111, projection='3d')
    # ax.scatter(g_x[:, 0], g_x[:, 1], g_x[:, 2])
    # plt.show()

    g_K = get_K()

    start = time.time()
    K_LU = scipy.linalg.cholesky(g_K, lower=True)
    assert np.allclose( g_K , np.dot(K_LU,K_LU.T) )
    end = time.time()
    if end - start > timer_thresh:
        print 'scipy.linalg.cholesky:', end - start, 'sec'

    start = time.time()
    K_LU2 = cholesky(g_K)
    assert np.allclose( K_LU2,K_LU )
    assert np.allclose( g_K , np.dot(K_LU2,K_LU2.T) )
    end = time.time()
    if end - start > timer_thresh:
        print 'cholesky:', end - start, 'sec'

#     K_LU4 = LU_decomp(g_K.copy())
#     for i in range(g_K.shape[0]):
#         print g_K[i,:]
#         print np.dot(K_LU4,K_LU4.T)[i,:]
#         print 'diff',np.max(np.abs(g_K[i,:] - np.dot(K_LU4,K_LU4.T)[i,:]))
#         assert np.allclose( g_K[i,:] , np.dot(K_LU4,K_LU4.T)[i,:] ,atol=.5,rtol=.5)
#     for i in range(K_LU4.shape[0]):
#         print K_LU4[i,:]
#         print K_LU2[i,:]
#         assert np.allclose( K_LU4[i,:] , K_LU2[i,:] ,atol=.1,rtol=.1)
#     assert np.allclose( g_K , np.dot(K_LU4,K_LU4.T)  ,atol=.5,rtol=.5)

    L = np.zeros(g_K.shape)
    K_LU3 = incomplete_LU_decomp(g_K,L)
#     assert np.allclose( g_K , np.dot(K_LU3,K_LU3.T),atol=.5,rtol=.5)
    print 'approx error',norm(g_K - np.dot(K_LU3,K_LU3.T))
    
    points_count *= 2
    if points_count > 10000:
        break



points_count 100
incomplete_LU_decomp: 0.00189089775085 sec
approx error 0.61646519295

points_count 200
incomplete_LU_decomp: 0.00487613677979 sec
approx error 0.697296027248

points_count 400
incomplete_LU_decomp: 0.0288088321686 sec
approx error 3.41360319729

points_count 800
incomplete_LU_decomp: 0.411829948425 sec
approx error 8.18446356884

points_count 1600
get_K: 0.239034891129 sec
scipy.linalg.cholesky: 0.519762992859 sec
cholesky: 0.476106882095 sec
incomplete_LU_decomp: 3.58400678635 sec
approx error 13.1400322501

points_count 3200
get_K: 0.904974937439 sec
scipy.linalg.cholesky: 4.13751292229 sec
cholesky: 4.82367086411 sec
incomplete_LU_decomp: 28.4523310661 sec
approx error 55.5271265636

points_count 6400
get_K: